<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [21]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "E:\\nutstore\\2019bf\\github\\Spinal-bootcamp/source/load-spinal.sc"

## Chapter


In [ ]:
class T2  extends Component{ 
    val a = out Bits(9 bit)
  a := B(32,9 bit)
}
showRtl(new T2)

In [ ]:
class Reg32(Addr: Long)  extends Component{ 
    val bmi = new Bundle{
    val rd,wr = in Bool()
    val addr  = in UInt(16 bits)    
    val wdata = in Bits(32 bit)
    val rdata = out(Reg(Bits(32 bit)) init 0 )}
    val clc   = in Bool()
    val hit   = bmi.addr === U(Addr)

     //val b = in(RO(2,0))
    //val c = in(RO(7,4))
    //val d = in(RO(7,8))
    val k = out(WRS(2 bits, 3 downto 2, B(0)))
    val p = out(WRS(2 bits, 1 downto 0, B(0)))
    val v = WRS(4 bits, 7 downto 4, B(3))
    val vo = Bits()
    when(clc){
        v := B(10,4 bits)
    }
    vo := v
    def RO(n: Int, ADDR: Int):Bits = {
        val ret = Bits(n bit)
        when(bmi.rd && bmi.addr === U(ADDR,16 bits)){            
        bmi.rdata := ret.resized
        }
        ret        
    }
    
    def WRS(bc: BitCount, section: Range, resetValue: Bits): Bits = {
      val ret = Reg(Bits(bc)) init resetValue
      val hit = (bmi.addr === U(0))
      when(bmi.wr && hit){
        ret := bmi.wdata(section)
      }.elsewhen(bmi.rd && hit){
        ret.setAll()
      }
      ret
    }
}
showRtl(new Reg32(321))

In [ ]:
class Reg32(Addr: Long)  extends Component{ 
    val bmi = new Bundle{
    val rd,wr = in Bool()
    val addr  = in UInt(16 bits)    
    val wdata = in Bits(32 bit)
    val rdata = out(Reg(Bits(32 bit)) init 0 )}
    val clc   = in Bool()
    val hit   = bmi.addr === U(Addr)

     //val b = in(RO(2,0))
  
    def RO(n: Int, ADDR: Int):Bits = {
        val ret = Bits(n bit)
        when(bmi.rd && bmi.addr === U(ADDR,16 bits)){            
        bmi.rdata := ret.resized
        }
        ret        
    }
    
 
}
showRtl(new Reg32(0xff))

In [ ]:
import scala.collection.mutable.ArrayBuffer
val a = new ArrayBuffer[Int]()
a += 1
a += (2, 3, 4, 5)
a   ++= Array(6, 7, 8, 9, 10)

In [ ]:
val fp = 10
val bc = 8 bits
val section: Range.Inclusive = fp+bc.value downto fp 
val c = 2 downto 1
val d = s"Section: ${section}"

In [ ]:
class Section(val max: Int, val min: Int){
  override def toString(): String = s"[${this.max}:${this.min}]"
}

object Section{
  def apply(rg: Range): Section = new Section(rg.max, rg.min)
}

object rangeToSection{
  implicit def tans(x: Range) = Section(x)
}

import rangeToSection._

val a: Section =   5 downto 2 

In [ ]:
val  a = ""
a.isEmpty
a.nonEmpty

In [ ]:
sealed trait AccessType

object AccessType {
  case object RO    extends AccessType  //- W: no effect, R: no effect
  case object RW    extends AccessType  //- W: as-is, R: no effect
  case object RC    extends AccessType  //- W: no effect, R: clears all bits
  case object RS    extends AccessType  //- W: no effect, R: sets all bits
  case object WRC   extends AccessType  //- W: as-is, R: clears all bits
  case object WRS   extends AccessType  //- W: as-is, R: sets all bits
  case object WC    extends AccessType  //- W: clears all bits, R: no effect
  case object WS    extends AccessType  //- W: sets all bits, R: no effect
  case object WSRC  extends AccessType  //- W: sets all bits, R: clears all bits
  case object WCRS  extends AccessType  //- W: clears all bits, R: sets all bits
  case object W1C   extends AccessType  //- W: 1/0 clears/no effect on matching bit, R: no effect
  case object W1S   extends AccessType  //- W: 1/0 sets/no effect on matching bit, R: no effect
  case object W1T   extends AccessType  //- W: 1/0 toggles/no effect on matching bit, R: no effect
  case object W0C   extends AccessType  //- W: 1/0 no effect on/clears matching bit, R: no effect
  case object W0S   extends AccessType  //- W: 1/0 no effect on/sets matching bit, R: no effect
  case object W0T   extends AccessType  //- W: 1/0 no effect on/toggles matching bit, R: no effect
  case object W1SRC extends AccessType  //- W: 1/0 sets/no effect on matching bit, R: clears all bits
  case object W1CRS extends AccessType  //- W: 1/0 clears/no effect on matching bit, R: sets all bits
  case object W0SRC extends AccessType  //- W: 1/0 no effect on/sets matching bit, R: clears all bits
  case object W0CRS extends AccessType  //- W: 1/0 no effect on/clears matching bit, R: sets all bits
  case object WO    extends AccessType  //- W: as-is, R: error
  case object WOC   extends AccessType  //- W: clears all bits, R: error
  case object WOS   extends AccessType  //- W: sets all bits, R: error
  case object W1    extends AccessType  //- W: first one after ~HARD~ reset is as-is, other W have no effects, R: no effect
  case object WO1   extends AccessType  //- W: first one after ~HARD~ reset is as-is, other W have no effects, R: error
  case object NA    extends AccessType  // -W: reserved, R: reserved
}


In [ ]:
val a = AccessType.NA
println(a)

In [ ]:
import $ivy.`com.lihaoyi::sourcecode:0.1.7`

In [10]:
trait BusSlaveFactory{
    val doWrite: Bool
    val doRead: Bool
    val bus: String
}
class ApbSlave(val bus: String) extends BusSlaveFactory{
    override val doWrite : Bool = False
    override val doRead: Bool = True
}

defined trait BusSlaveFactory
defined class ApbSlave

In [8]:
val a = 2
a.asInstanceOf[Long]
val b: Long = 2
a.asInstanceOf[Int]

a: Int = 2
res7_1: Long = 2L
b: Long = 2L
res7_3: Int = 2

In [4]:
class T2  extends Component{ 
    val a = out Bits(9 bit)
    a := B(32,9 bit)
    class Dog{}
    val xiaogou = new Dog
}
showRtl(new T2)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.27 13:04:03
[Progress] at 320.539 : Elaborate components
[Progress] at 320.541 : Checks and transforms
[Progress] at 320.544 : Generate Verilog
[Done] at 320.549
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 27/10/2019, 13:04:03
// Component : T2


module T2 (
      output [8:0] a);
  assign a = (9'b000100000);
endmodule




defined class T2

In [17]:
class T2  extends Component{ 
    //val a = out Bits(9 bit)
    //val c = out UInt()
    //a := B(32,9 bit)
    class Dog{
        def genTimer(n: Int) = {
            val timer = Reg(UInt(n bits)) init 0 
            val clearTimer = in Bool()
            when(clearTimer){
                timer init 0
            }.otherwise {
                timer := timer + 1
            }
            (clearTimer,timer)
        }
    }
    val xiaogou = new Dog
//xiaogou.genTimer(8)
   val   (weigou,timer) = xiaogou.genTimer(8)
    //c : = d
}
showRtl(new T2)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.27 13:40:01
[Progress] at 2477.798 : Elaborate components
[Progress] at 2477.803 : Checks and transforms
[Progress] at 2477.807 : Generate Verilog
[Warning] 4 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 2477.810
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 27/10/2019, 13:40:01
// Component : T2


module T2 (
      input   weigou,
      input   clk,
      input   reset);
  reg [7:0] timer;
  always @ (posedge clk or posedge reset) begin
    if (reset) begin
      timer <= (8'b00000000);
      if(weigou)begin
        timer <= (8'b00000000);
      end
    end else begin
      if(! weigou) begin
        timer <= (timer + (8'b00000001));
      end
    end
  end

endmodule




defined class T2

### get config from GlobalData

In [26]:
import spinal.core.internals._

class T2  extends Component{ 
  val pc = GlobalData.get.phaseContext
  val a = in Bits(8 bit)
  println(GlobalData.get.commonClockConfig.clockEdge)
  println(pc.config)
  println(pc.config.targetDirectory)
}
 showRtl(new T2)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.28 11:37:03
[Progress] at 81500.491 : Elaborate components
spinal.core.RISING$@653eed30
SpinalConfig(spinal.core.Verilog$@725c06a8,Set(),Set(),false,ClockDomainConfig(spinal.core.RISING$@653eed30,spinal.core.ASYNC$@3e5ce588,spinal.core.HIGH$@54f02e0d,spinal.core.HIGH$@54f02e0d,spinal.core.HIGH$@54f02e0d),false,UnknownFrequency(),rtl/,false,null,null,,false,false,null,Device(?,?,?),false,true,false,false,false,false,false,true,false,ArrayBuffer(),ArrayBuffer(),ArrayBuffer(spinal.core.internals.PhaseMemBlackBoxingDefault@4a9268c5),null,true)
rtl/
[Progress] at 81500.493 : Checks and transforms
[Progress] at 81500.494 : Generate Verilog
[Warning] 1 signals were pruned. You can call printPruned on the backend report to get more informations.
[Done] at 81500.496
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc

import spinal.core.internals._


defined class T2

In [28]:
val a = List(1,2,3,4,5)
a.tails

a: List[Int] = List(1, 2, 3, 4, 5)
res27_1: Iterator[List[Int]] = non-empty iterator

In [33]:
val a = List(1)
a.tail
a.tail.last

: 

In [35]:
val a = "class RegIfExample3"
a.split(" ").last

a: String = "class RegIfExample3"
res34_1: String = "RegIfExample3"